# Paquetes a utilizar

In [ ]:
import sklearn as skl
import pandas as pd
import numpy as np
import xgboost as xgb
from boruta import BorutaPy

# Base de datos

In [ ]:
df = pd.read_csv("data/wisconsin_breast_cancer_dataset.csv")
df = df.dropna()
df = df.rename(columns={'diagnosis':'Label'})
df['Label'].value_counts()

In [ ]:
df.describe().T

In [ ]:
df.dtypes

# Variable dependiente que debe predecirse y codificación de datos categóricos

In [ ]:
y = df["Label"].values
Y = skl.preprocessing.LabelEncoder().fit_transform(y)

# Definir x, normalizar valores y definir variables independientes

In [ ]:
X = df.drop(labels = ["Label", "id"], axis=1) 
nombres_de_funciones = np.array(X.columns)
scaler = skl.preprocessing.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Train and test para verificar la precisión después de ajustar el modelo

In [ ]:
X_train, X_test, y_train, y_test = skl.model_selection.train_test_split(X, Y, test_size=0.25, random_state=42)

# XGBOOST para ser utilizado por Boruta

In [ ]:
modelo = xgb.XGBClassifier()

- Crear funciones de sombra: funciones aleatorias y valores aleatorios en columnas
- Entrenar Random Forest / XGBoost y calcular la importancia de la característica a través de la disminución media de la impureza
- Comprobar si las características reales tienen mayor importancia en comparación con las características de sombra
- Repetir esto para cada iteración
- Si la función original funcionó mejor, marcarla como importante.

In [ ]:
# Especificar cómo se determinan las características relevantes utilizando el algoritmo Boruta
selector = BorutaPy(modelo, n_estimators='auto', verbose=2, random_state=1)
# Hallar todas las características importantes
selector.fit(X_train, y_train)
# Aplicar el método transform() en el conjunto de datos X para limitarlo a solo las características seleccionadas
X_filtered = selector.transform(X_train)  # Utilizar técnicas de selección de características y obtener el conjunto de datos modificado

# zip nombres de características, rangos y decisiones

In [ ]:
feature_ranks = list(zip(nombres_de_funciones, 
                         selector.ranking_, 
                         selector.support_))


# Resultados

In [ ]:
for feat in feature_ranks:
    print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))

In [ ]:
# Utilizando un conjunto específico de características, ajustar un modelo XGBoost en los datos de entrenamiento
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_filtered, y_train)
# Emplear el modelo previamente entrenado para hacer predicciones con el conjunto de datos de prueba
# Antes de hacer las predicciones, aplicar las mismas técnicas de selección de características en los datos de prueba para asegurarse de que solo se estén utilizando las características relevantes
X_test_filtered = selector.transform(X_test)
prediction_xgb = xgb_model.predict(X_test_filtered)

## Precisión

In [ ]:
skl.metrics.accuracy_score(y_test, prediction_xgb)